# π Estimation with Monte Carlo methods
We demonstrate how to run Monte Carlo simulations with lithops over GCP Cloud Functions and AWS Lambda Functions. This notebook contains an example of estimation the number π with Monte Carlo. The goal of this notebook is to demonstrate how loud Functions can benefit Monte Carlo simulations and not how it can be done using lithops.<br>
A Monte Carlo algorithm would randomly place points in the square and use the percentage of randomized points inside of the circle to estimate the value of π
![pi](https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif)

Requirements to run this notebook:
* AWS Cloud or GCP account. 
* You will need to have at least one existing object storage bucket. 
* Lithops 3.1.0

# Step 1 - Install dependencies
Install dependencies

In [1]:
from time import time
from random import random
import logging
import numpy as np
import sys

try:
    import lithops
except:
    !{sys.executable} -m pip install lithops
    import lithops

# you can modify logging level if needed
#logging.basicConfig(level=logging.INFO)

# Step 2 - Write Python code that implements Monte Carlo simulation 
Below is an example of Python code to demonstrate Monte Carlo model for estimate PI

'EstimatePI' is a Python class that we use to represent a single PI estimation. You may configure the following parameters:

MAP_INSTANCES - number of IBM Cloud Function invocations. Default is 100<br>
randomize_per_map - number of points to random in a single invocation. Default is 10,000,000

Our code contains two major Python methods:

def randomize_points(self,data=None) - a function to random number of points and return the percentage of points
    that inside the circle<br>
def process_in_circle_points(self, results, futures): - summarize results of all randomize_points
  executions (aka "reduce" in map-reduce paradigm)

In [2]:
MAP_INSTANCES = 25


class EstimatePI:
    randomize_per_map = 100000

    def __init__(self):
        self.total_randomize_points = MAP_INSTANCES * self.randomize_per_map

    def __str__(self):
        return "Total Randomize Points: {:,}".format(self.randomize_per_map * MAP_INSTANCES)

    @staticmethod
    def predicate():
        x = random()
        y = random()
        return (x ** 2) + (y ** 2) <= 1

    def randomize_points(self, data):
        in_circle = 0
        for _ in range(self.randomize_per_map):
            in_circle += self.predicate()
        return float(in_circle / self.randomize_per_map)

    def process_in_circle_points(self, results):
        in_circle_percent = 0
        for map_result in results:
            in_circle_percent += map_result
        estimate_PI = float(4 * (in_circle_percent / MAP_INSTANCES))
        return estimate_PI

# Step 3 - Configure access to your Cloud Storage and Cloud Functions

Configure access details to your AWS or GCP Cloud Functions.  'storage_bucket'  should point to some pre-existing bucket. This bucket will be used by Lithops to store intermediate results. All results will be stored in the folder `lithops.jobs`. For additional configuration parameters see [configuration section](https://github.com/lithops-cloud/lithops)

For GCP your `.lithops_config` should be similar to: 
    
    lithops:
        storage: gcp_storage
        backend: gcp_functions
        bucket: lithops-pipelines
    
    gcp:
        credentials_path : <PATH_TO_JSON_KEYS>
        region : <GCP_REGION>
    
    gcp_functions:
        region : <GCP_REGION>
    
    gcp_storage:
        region: <GCP_REGION>
        storage_bucket: <GCP_STORAGE_BUCKET>

For AWS your `.lithops_config` should be similar to: 
    
    lithops:
        storage: aws_s3
        backend: aws_lambda
    
    aws:
        access_key_id : <AWS_ACCESS_KEY_ID>
        secret_access_key : <AWS_SECRET_ACCESS_KEY> 
        
    aws_s3:
        storage_bucket: <S3_BUCKET>
        region_name : <REGION>
    
    aws_lambda:
        execution_role: <AWS_ROLE_ARN>
        region_name: <REGION>

# Step 4 - Execute simulation with Lithops over AWS Lambda or Google Cloud Functions 

In [4]:
iterdata = [0] * MAP_INSTANCES
est_pi = EstimatePI()

start_time = time()
print("Monte Carlo simulation for estimating PI spawing over {} IBM Cloud Function invocations".format(MAP_INSTANCES))
# obtain lithops executor
pw = lithops.FunctionExecutor(runtime_memory=4096)
# execute the code
pw.map_reduce(est_pi.randomize_points, iterdata, est_pi.process_in_circle_points)
#get results
result = pw.get_result()
elapsed = time()
print(str(est_pi))
print("Estimation of Pi: ", result)
print("\nCompleted in: " + str(elapsed - start_time) + " seconds")

2023-11-02 13:04:48,749 [INFO] config.py:134 -- Lithops v2.9.0
2023-11-02 13:04:48,827 [INFO] gcp_storage.py:45 -- Google Cloud Storage client created


Monte Carlo simulation for estimating PI spawing over 25 IBM Cloud Function invocations


2023-11-02 13:04:48,990 [INFO] gcp_functions.py:68 -- Google Cloud Functions client created - Region: us-east1 - Project: pipelines-serverless
2023-11-02 13:04:48,993 [INFO] invokers.py:108 -- ExecutorID 821ea9-1 | JobID M000 - Selected Runtime: default-runtime-v39 - 4096MB
2023-11-02 13:04:49,984 [INFO] invokers.py:172 -- ExecutorID 821ea9-1 | JobID M000 - Starting function invocation: randomize_points() - Total: 25 activations
2023-11-02 13:04:50,005 [INFO] invokers.py:208 -- ExecutorID 821ea9-1 | JobID M000 - View execution logs at /tmp/lithops-jordi/logs/821ea9-1-M000.log
2023-11-02 13:04:50,008 [INFO] wait.py:97 -- ExecutorID 821ea9-1 - Waiting for 20% of 25 function activations to complete


    0%|          | 0/5  

2023-11-02 13:04:57,088 [INFO] invokers.py:108 -- ExecutorID 821ea9-1 | JobID R000 - Selected Runtime: default-runtime-v39 - 4096MB
2023-11-02 13:04:58,572 [INFO] invokers.py:172 -- ExecutorID 821ea9-1 | JobID R000 - Starting function invocation: process_in_circle_points() - Total: 1 activations
2023-11-02 13:04:58,577 [INFO] invokers.py:208 -- ExecutorID 821ea9-1 | JobID R000 - View execution logs at /tmp/lithops-jordi/logs/821ea9-1-R000.log
2023-11-02 13:04:58,584 [INFO] wait.py:97 -- ExecutorID 821ea9-1 - Getting results from 26 function activations


    0%|          | 0/26  

2023-11-02 13:05:06,682 [INFO] executors.py:612 -- ExecutorID 821ea9-1 - Cleaning temporary data


Total Randomize Points: 2,500,000
Estimation of Pi:  3.1420799999999987

Completed in: 17.958427906036377 seconds
